In [ ]:
import json, os, lxml.html
from urllib.parse import urljoin   # only needed if you want to absolutize links

URL = "https://github.com/facebook"

FOLDER = URL.rstrip("/").replace("://","_").replace("/","_")
HTML_PATH     = os.path.join(FOLDER, "page.html")        
JSON_PATH     = os.path.join(FOLDER, "segmentation.json") 
OUT_PATH      = os.path.join(FOLDER, "annotated.html")

https://kubernetes.io/docs/concepts/


In [2]:
def add_style(el, css):
    """Append CSS to an element’s existing inline style."""
    current = el.get("style", "")
    if current and not current.endswith(";"):
        current += ";"
    el.set("style", current + css)

# 1) Fiel Setup
with open(HTML_PATH, encoding="utf‑8") as f:
    html_text = f.read()

root = lxml.html.fromstring(html_text)
if root.find(".//head") is None:
    root.insert(0, lxml.html.Element("head"))

with open(JSON_PATH, encoding="utf‑8") as f:
    segmentation = json.load(f)

# 2) outline parents & leaves
for parent_xpath, info in segmentation.items():
    for el in root.xpath(parent_xpath):
        add_style(el, "outline:4px solid red;")

    for leaf_xpath in info.get("leaves", []):
        try:
            for el in root.xpath(leaf_xpath):
                add_style(el, "outline:2px solid lime;")
        except lxml.etree.XPathEvalError:
            pass  # skip malformed XPath silently

# (optional) make relative links absolute so images/CSS resolve when opened locally
root.make_links_absolute(URL)

# ---------------------------------------------------------------- 3) save annotated HTML
os.makedirs(FOLDER, exist_ok=True)
with open(OUT_PATH, "wb") as f:
    f.write(
        lxml.html.tostring(
            root,
            pretty_print=True,
            encoding="utf-8",
            doctype="<!DOCTYPE html>",
        )
    )

print(f"✅ Annotated page written to {OUT_PATH}")


✅ Annotated page written to https_kubernetes.io_docs_concepts/annotated.html
